In [10]:
import requests
import re
import pandas as pd
import numpy as np

In [ ]:
# webscraping: getting table of soccer players from wiki
headers = {'User-Agent': 'For Stat386 homework'}
url = "https://en.wikipedia.org/wiki/List_of_men%27s_footballers_with_100_or_more_international_caps"
r = requests.get(url, headers=headers)
tables = pd.read_html(r.text)
df = tables[1]
df.head()

C:\Users\Owner\AppData\Local\Temp\ipykernel_23680\3738025927.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


,Rank,Players,Nation,Confederation,Caps,Debut,Latest
0,1,Cristiano Ronaldo,Portugal,UEFA,226,20 August 2003,13 November 2025
1,2,Bader Al-Mutawa,Kuwait,AFC,202,4 September 2003,14 June 2022
2,3,Lionel Messi,Argentina,CONMEBOL,196,17 August 2005,14 November 2025
3,4,Soh Chin Ann[a],Malaysia,AFC,195,19 November 1969,18 October 1984
4,5,Luka Modrić,Croatia,UEFA,192,1 March 2006,12 October 2025


In [41]:
df['Players'] = df['Players'].str.replace(r'\[([a-z]+)\]', '', regex=True)
df['Debut'] = pd.to_datetime(df['Debut'], errors='coerce')
df['Latest'] = pd.to_datetime(df['Latest'], errors='coerce')
df["Years Active"] = df['Latest'].dt.year - df['Debut'].dt.year
df.head()

,Rank,Players,Nation,Confederation,Caps,Debut,Latest,Years Active
0,1,Cristiano Ronaldo,Portugal,UEFA,226,2003-08-20,2025-11-13,22
1,2,Bader Al-Mutawa,Kuwait,AFC,202,2003-09-04,2022-06-14,19
2,3,Lionel Messi,Argentina,CONMEBOL,196,2005-08-17,2025-11-14,20
3,4,Soh Chin Ann,Malaysia,AFC,195,1969-11-19,1984-10-18,15
4,5,Luka Modrić,Croatia,UEFA,192,2006-03-01,2025-10-12,19
